In [26]:
import pandas as pd
from fbprophet import Prophet

In [27]:

df = pd.read_csv('../data/lrpd-clean.csv')
df['INCIDENT_DATE'] = pd.to_datetime(df['INCIDENT_DATE'])
df2 = df.groupby(pd.Grouper(key='INCIDENT_DATE', freq='D')).size().reset_index(name='INCIDENT_COUNT')
df2.columns = ['ds', 'y']
df2.tail()

,ds,y
2112,2022-10-14,49
2113,2022-10-15,28
2114,2022-10-16,31
2115,2022-10-17,47
2116,2022-10-18,35


In [28]:
#Get all holidays in US as dataframe from 2017 to 2022
import holidays

us_holidays = holidays.UnitedStates(years=[2017, 2018, 2019, 2020, 2021, 2022, 2023])
us_holidays = pd.DataFrame.from_dict(us_holidays, orient='index').reset_index()
us_holidays.columns = ['ds', 'holiday']
us_holidays['ds'] = pd.to_datetime(us_holidays['ds'])


In [29]:
#let train be 80% of the data
train = df2[:int(0.8 * (len(df2)))]
test = df2[int(0.8 * (len(df2))):]
train.tail(), test.head()

(             ds   y
 1688 2021-08-16  50
 1689 2021-08-17  44
 1690 2021-08-18  39
 1691 2021-08-19  33
 1692 2021-08-20  51,
              ds   y
 1693 2021-08-21  40
 1694 2021-08-22  31
 1695 2021-08-23  57
 1696 2021-08-24  40
 1697 2021-08-25  53)

In [30]:

from statsmodels.tools.eval_measures import rmse
import itertools

param_grid = {
    'changepoint_prior_scale': [0.001, 0.01, 0.1, 0.5],
    'seasonality_prior_scale': [0.01, 0.1, 1.0, 10.0],
    'holidays_prior_scale': [0.01, 0.1, 1.0, 10.0],
    'seasonality_mode': ['additive', 'multiplicative'],
}

all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]
rmses = []

for params in all_params:
    m = Prophet(**params,
                holidays=us_holidays,
                yearly_seasonality=True,
                ).fit(train)
    future = m.make_future_dataframe(periods=test.shape[0])
    forecast = m.predict(future)
    predictions = forecast[int(0.8 * (len(forecast))):]['yhat']
    predictions = predictions[:len(test)]
    actual = test['y']
    rmses.append(rmse(actual, predictions))

tuning_results = pd.DataFrame(all_params)
tuning_results['rmse'] = rmses
tuning_results

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=Tru

,changepoint_prior_scale,seasonality_prior_scale,holidays_prior_scale,seasonality_mode,rmse
0,0.001,0.01,0.01,additive,10.100906
1,0.001,0.01,0.01,multiplicative,10.402433
2,0.001,0.01,0.10,additive,10.121739
3,0.001,0.01,0.10,multiplicative,10.454130
4,0.001,0.01,1.00,additive,10.129796
...,...,...,...,...,...
123,0.500,10.00,0.10,multiplicative,10.013961
124,0.500,10.00,1.00,additive,10.520265
125,0.500,10.00,1.00,multiplicative,10.075205
126,0.500,10.00,10.00,additive,10.519870
